In [8]:
import requests

COVALENT_API_KEY = "cqt_"


def get_compound_v2_activity(wallet_address):
    a=0x0039f22efb07a647557c7c5d17854cfd6d489ef3
    url = f"https://api.covalenthq.com/v1/1/address/{a}/transactions_v2/?key={COVALENT_API_KEY}"
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json().get("data", {}).get("items", [])
        return data
    else:
        print(f"❌ Failed for {wallet_address}: {response.status_code}")
        return []


def compute_features_covalent(items):
    deposit = borrow = repay = redeem = 0

    for tx in items:
        for log in tx.get("log_events", []):
            decoded = log.get("decoded")
            if not decoded:
                continue

            event_name = decoded.get("name", "")
            params = decoded.get("params")
            if not params:
                continue

            value = 0
            for param in params:
                if param.get("name") in {"amount", "mintAmount", "borrowAmount", "repayAmount", "redeemAmount", "value"}:
                    try:
                        value = float(param.get("value", 0)) / 1e18  # Normalize for 18 decimals
                    except:
                        value = 0

            if event_name == "Mint":
                deposit += value
            elif event_name == "Borrow":
                borrow += value
            elif event_name == "RepayBorrow":
                repay += value
            elif event_name == "Redeem":
                redeem += value

    net_debt = max(0, borrow - repay)
    repay_ratio = repay / (borrow + 1)
    redeem_ratio = redeem / (deposit + 1)

    return {
        "total_deposit_usd": deposit,
        "total_borrow_usd": borrow,
        "total_repay_usd": repay,
        "repay_to_borrow_ratio": repay_ratio,
        "redeem_to_deposit_ratio": redeem_ratio,
        "liquidation_count": 0,
        "net_debt": net_debt
    }




def compute_score(f):
    score = 500
    score += min(f["total_deposit_usd"] / 100, 200)
    score += min(f["repay_to_borrow_ratio"] * 100, 200)
    score += min(f["redeem_to_deposit_ratio"] * 100, 100)
    score -= min(f["liquidation_count"] * 100, 200)
    score -= min(f["net_debt"] / 100, 100)
    return int(max(0, min(1000, score)))

import pandas as pd

def process_wallets_from_csv(csv_file):
    df = pd.read_csv(csv_file)
    wallet_list = df["wallet_id"].tolist()
    results = []

    for wallet in wallet_list:
        print(f"🔍 Fetching: {wallet}")
        items = get_compound_v2_activity(wallet)
        if items:
            features = compute_features_covalent(items)
            score = compute_score(features)
            results.append({"wallet_id": wallet, "score": score})
        else:
            print(f"⚠️ No data for {wallet}")

    return pd.DataFrame(results)


if __name__ == "__main__":
    output_df = process_wallets_from_csv("Wallet_id.csv")
    output_df.to_csv("wallet_scores.csv", index=False)
    print("✅ Done. Scores saved to wallet_scores.csv")


🔍 Fetching: 0x0039f22efb07a647557c7c5d17854cfd6d489ef3
🔍 Fetching: 0x06b51c6882b27cb05e712185531c1f74996dd988
🔍 Fetching: 0x0795732aacc448030ef374374eaae57d2965c16c
🔍 Fetching: 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
🔍 Fetching: 0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
🔍 Fetching: 0x104ae61d8d487ad689969a17807ddc338b445416
🔍 Fetching: 0x111c7208a7e2af345d36b6d4aace8740d61a3078
🔍 Fetching: 0x124853fecb522c57d9bd5c21231058696ca6d596
🔍 Fetching: 0x13b1c8b0e696aff8b4fee742119b549b605f3cbc
🔍 Fetching: 0x1656f1886c5ab634ac19568cd571bc72f385fdf7
🔍 Fetching: 0x1724e16cb8d0e2aa4d08035bc6b5c56b680a3b22
🔍 Fetching: 0x19df3e87f73c4aaf4809295561465b993e102668
🔍 Fetching: 0x1ab2ccad4fc97c9968ea87d4435326715be32872
🔍 Fetching: 0x1c1b30ca93ef57452d53885d97a74f61daf2bf4f
🔍 Fetching: 0x1e43dacdcf863676a6bec8f7d6896d6252fac669
🔍 Fetching: 0x22d7510588d90ed5a87e0f838391aaafa707c34b
🔍 Fetching: 0x24b3460622d835c56d9a4fe352966b9bdc6c20af
🔍 Fetching: 0x26750f1f4277221bdb5f6991473c6ece8c821f9d
🔍 Fetching

In [2]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.
